# Reading Comments from Docx

## 1. [Cross Platform Solution]

Reference:

1. https://stackoverflow.com/questions/47390928/extract-docx-comments

In [1]:
from docx import Document
from lxml import etree
import zipfile

ooXMLns = {'w':'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

#### 1.1 Get Comment and Reference Paragraph

In [2]:
#Function to extract all the comments of document(Same as accepted answer)
#Returns a dictionary with comment id as key and comment string as value
def get_document_comments(docxFileName):
    comments_dict={}
    docxZip = zipfile.ZipFile(docxFileName)
    commentsXML = docxZip.read('word/comments.xml')
    et = etree.XML(commentsXML)
    comments = et.xpath('//w:comment',namespaces=ooXMLns)
    for c in comments:
        comment=c.xpath('string(.)',namespaces=ooXMLns)
        comment_id=c.xpath('@w:id',namespaces=ooXMLns)[0]
        comments_dict[comment_id]=comment
    docxZip.close()
    return comments_dict

#Function to fetch all the comments in a paragraph
def paragraph_comments(paragraph,comments_dict):
    comments=[]
    for run in paragraph.runs:
        comment_reference=run._r.xpath("./w:commentReference")
        if comment_reference:
            comment_id=comment_reference[0].xpath('@w:id',namespaces=ooXMLns)[0]
            comment=comments_dict[comment_id]
            comments.append(comment)
    return comments

#Function to fetch all comments with their referenced paragraph
#This will return list like this [{'Paragraph text': [comment 1,comment 2]}]
def comments_with_reference_paragraph(docxFileName):
    document = Document(docxFileName)
    comments_dict=get_document_comments(docxFileName)
    comments_with_their_reference_paragraph=[]
    for paragraph in document.paragraphs:  
        if comments_dict: 
            comments=paragraph_comments(paragraph,comments_dict)  
            if comments:
                comments_with_their_reference_paragraph.append({paragraph.text: comments})
    return comments_with_their_reference_paragraph

In [3]:
file_name = 'comment_test_1.docx' #filepath for the input document
print(comments_with_reference_paragraph(file_name)[0])

{"Bangladesh (/ˌbæŋɡləˈdɛʃ, ˌbɑːŋ-/;[12] Bengali: বাংলাদেশ, pronounced [ˈbaŋlaˌdeʃ] (listen)), officially the People's Republic of Bangladesh, is a country in South Asia. It is the eighth-most populous country in the world, with a population exceeding 165 million people in an area of 148,460 square kilometres (57,320 sq mi).[7] Bangladesh is among the most densely populated countries in the world, and shares land borders with India to the west, north, and east, and Myanmar to the southeast; to the south it has a coastline along the Bay of Bengal. It is narrowly separated from Bhutan and Nepal by the Siliguri Corridor; and from China by the Indian state of Sikkim in the north. Dhaka, the capital and largest city, is the nation's political, financial and cultural centre. Chittagong, the second-largest city, is the busiest port on the Bay of Bengal. The official language is Bengali, one of the easternmost branches of the Indo-European language family.": ['Full Name']}


In [4]:
get_document_comments(file_name)

{'0': 'Full Name', '1': 'First Muslim Rulling', '2': 'Bengali Majority'}

#### 1.2 Get Coomment Author, Comment Date and Comment

In [5]:
from lxml import etree
import zipfile

ooXMLns = {'w':'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

def get_comments(docxFileName):
    docxZip = zipfile.ZipFile(docxFileName)
    commentsXML = docxZip.read('word/comments.xml')
    et = etree.XML(commentsXML)
    comments = et.xpath('//w:comment',namespaces=ooXMLns)
    for c in comments:
        # attributes:
        print(c.xpath('@w:author',namespaces=ooXMLns))
        print(c.xpath('@w:date',namespaces=ooXMLns))
        # string value of the comment:
        print(c.xpath('string(.)',namespaces=ooXMLns))
        
        
get_comments('comment_test_1.docx')

['Md Abul Bashar']
['2022-12-04T07:32:00Z']
Full Name
['Md Abul Bashar']
['2022-12-04T07:34:00Z']
First Muslim Rulling
['Md Abul Bashar']
['2022-12-04T07:36:00Z']
Bengali Majority


## 2. [MS Windows Solution]

Reference: 

1. https://stackoverflow.com/questions/47390928/extract-docx-comments
2. https://stackoverflow.com/questions/21898656/why-do-i-get-an-error-when-iterating-over-paragraphs-in-ms-word-with-win32com

#### 2.1 Get Comments, Author and Reference text

In [6]:
import win32com.client as win32
from win32com.client import constants

word = win32.gencache.EnsureDispatch('Word.Application')
word.Visible = False 
filepath = "C:\\Users\\basharm\\OneDrive - Queensland University of Technology\\JupyterPythonQUT\\DoR\\Stage2\\comment_test_1.docx"

def get_comments(filepath):
    doc = word.Documents.Open(filepath) 
    doc.Activate()
    activeDoc = word.ActiveDocument
    for c in activeDoc.Comments: 
        if c.Ancestor is None: #checking if this is a top-level comment
            print("Comment by: " + c.Author)
            print("Comment text: " + c.Range.Text) #text of the comment
            print("Regarding: " + c.Scope.Text) #text of the original document where the comment is anchored 
            if len(c.Replies)> 0: #if the comment has replies
                print("Number of replies: " + str(len(c.Replies)))
                for r in range(1, len(c.Replies)+1):
                    print("Reply by: " + c.Replies(r).Author)
                    print("Reply text: " + c.Replies(r).Range.Text) #text of the reply
            print()
    doc.Close()
    
get_comments(filepath)

Comment by: Md Abul Bashar
Comment text: Full Name
Regarding: People's Republic of Bangladesh

Comment by: Md Abul Bashar
Comment text: First Muslim Rulling
Regarding: The Muslim conquest of Bengal began in 1204 when Bakhtiar Khalji overran northern Bengal and invaded Tibet.

Comment by: Md Abul Bashar
Comment text: Bengali Majority
Regarding: Bengalis make up 99% of the total population of Bangladesh.



#### 2.2 Get paragraphs

In [7]:
def get_paragraphs(filepath):
    doc = word.Documents.Open(filepath) 
    doc.Activate()
    activeDoc = word.ActiveDocument
    for paragraph in activeDoc.Paragraphs:
        print (paragraph.Range.Text)
        print()
    doc.Close()
    
get_paragraphs(filepath)

Introduction

Bangladesh (/ˌbæŋɡləˈdɛʃ, ˌbɑːŋ-/;[12] Bengali: বাংলাদেশ, pronounced [ˈbaŋlaˌdeʃ] (listen)), officially the People's Republic of Bangladesh, is a country in South Asia. It is the eighth-most populous country in the world, with a population exceeding 165 million people in an area of 148,460 square kilometres (57,320 sq mi).[7] Bangladesh is among the most densely populated countries in the world, and shares land borders with India to the west, north, and east, and Myanmar to the southeast; to the south it has a coastline along the Bay of Bengal. It is narrowly separated from Bhutan and Nepal by the Siliguri Corridor; and from China by the Indian state of Sikkim in the north. Dhaka, the capital and largest city, is the nation's political, financial and cultural centre. Chittagong, the second-largest city, is the busiest port on the Bay of Bengal. The official language is Bengali, one of the easternmost branches of the Indo-European language family.



History

Bangladesh fo